# Introduction to GCP and Semantic Kernel

### Google Cloud Platform (GCP)
* Google Cloud Platform (GCP) is a suite of cloud computing services offered by Google, providing a range of infrastructure, platform, and serverless computing environments.
* It includes services like Google Cloud Storage (used in this notebook for file management), Compute Engine, BigQuery, and AI/ML tools. GCP enables scalable, secure, and efficient data storage, processing, and analysis, making it ideal for building and deploying applications, managing data, and integrating AI capabilities.

### Semantic Kernel
* Semantic Kernel is an open-source SDK developed by Microsoft that simplifies the integration of AI models and services into applications. * It acts as an orchestration layer, allowing developers to combine AI capabilities (like large language models from Gemini) with traditional programming logic. It supports plugins, agents, and workflows, enabling the creation of intelligent applications that can process natural language, automate tasks, and integrate with external services like GCP.

### Semantic Kernel with GCP
* This notebook demonstrates the integration of Semantic Kernel with Google Cloud Platform (GCP) to create a multi-agent system for file management, analysis, and reporting.
* Below are the code cells and their outputs, as provided in the original notebook.

## Cell 1: Install Required Libraries

In [1]:
!pip install semantic-kernel google-generativeai google-cloud-storage python-dotenv -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires pyee<12.0.0,>=11.0.0, but you have pyee 13.0.0 which is incompatible.
pyppeteer 2.0.0 requires websockets<11.0,>=10.0, but you have websockets 15.0.1 which is incompatible.


## Cell 2: Import Libraries

In [5]:
import os
import semantic_kernel as sk
import google.generativeai as genai
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from google.cloud import storage
from pathlib import Path

print("✅ All libraries imported")

✅ All libraries imported


## Cell 3: Set Up Environment and Initialize Semantic Kernel

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "your_iam_crediential_role_json_file"

# Gemini Setup
GEMINI_API_KEY = "your_gemini_appi_key"
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)

# Test Gemini connection
try:
    model = genai.GenerativeModel('gemini-1.5-flash')
    test_response = model.generate_content("Hello, test connection")
    print(f"✅ Gemini 1.5 Flash connected: {test_response.text[:50]}...")
except Exception as e:
    print(f"❌ Gemini connection failed: {str(e)}")

✅ Gemini 1.5 Flash connected: Connection successful!  How can I help you today?
...


## Cell 4: Define Enhanced GCP Agent

In [7]:
class GeminiChatCompletion:
    """Custom Gemini Chat Completion for Semantic Kernel"""

    def __init__(self, api_key, model_name="gemini-1.5-flash"):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name)
        self.service_id = "gemini"

    async def complete_chat_async(self, messages, **kwargs):
        """Complete chat using Gemini"""
        try:
            # Convert messages to prompt
            prompt_parts = []
            for message in messages:
                if hasattr(message, 'role') and hasattr(message, 'content'):
                    role = message.role
                    content = message.content
                elif isinstance(message, dict):
                    role = message.get('role', '')
                    content = message.get('content', '')
                else:
                    content = str(message)
                    role = 'user'

                if role == 'system':
                    prompt_parts.append(f"System: {content}")
                elif role == 'user':
                    prompt_parts.append(f"User: {content}")
                elif role == 'assistant':
                    prompt_parts.append(f"Assistant: {content}")
                else:
                    prompt_parts.append(content)

            full_prompt = "\n".join(prompt_parts)

            # Generate response
            response = self.model.generate_content(full_prompt)

            # Return in expected format
            class ChatMessage:
                def __init__(self, content):
                    self.content = content
                    self.role = "assistant"

            return ChatMessage(response.text)

        except Exception as e:
            print(f"Gemini completion error: {str(e)}")
            class ErrorMessage:
                def __init__(self, error):
                    self.content = f"Error: {str(error)}"
                    self.role = "assistant"
            return ErrorMessage(e)

# Create Kernel with Gemini
kernel = sk.Kernel()

# Add Gemini service to kernel
gemini_service = GeminiChatCompletion(GEMINI_API_KEY)
kernel.add_service(gemini_service)

print("✅ Kernel ready with Gemini 1.5 Flash")

✅ Kernel ready with Gemini 1.5 Flash


In [14]:
class EnhancedGCPAgent:
    """Enhanced GCP Agent with better file handling"""

    def __init__(self):
        self.client = storage.Client(project="Gen-ai")  # Your project name
        self.bucket_name = "bucket_demo8"  # Your bucket name
        self.base_dir = Path("/content")  # Your file path

    @kernel_function(description="List files in bucket and local directory")
    def list_files(self) -> str:
        """List files in bucket and local directory"""
        try:
            # List files in bucket
            bucket = self.client.bucket(self.bucket_name)
            bucket_files = [blob.name for blob in bucket.list_blobs()]

            # List files in local directory
            local_files = []
            if self.base_dir.exists():
                local_files = [f.name for f in self.base_dir.iterdir() if f.is_file()]

            return f"Bucket files: {bucket_files}\\nLocal files: {local_files}"
        except Exception as e:
            return f"Error listing files: {str(e)}"

    @kernel_function(description="Upload file to bucket with smart filename matching")
    def upload_file(self, filename: str) -> str:
        """Upload file with smart filename matching"""
        try:
            # Try multiple filename variations
            possible_names = [
                filename,
                filename.replace('_', ' '),
                filename.replace(' ', '_'),
                filename.replace('eer2', 'eer_2'),
                filename.replace('eer_2', 'eer2')
            ]

            file_path = None
            actual_filename = None

            # Find the actual file
            for name in possible_names:
                test_path = self.base_dir / name
                if test_path.exists():
                    file_path = test_path
                    actual_filename = name
                    break

            if file_path is None:
                # List available files for user
                if self.base_dir.exists():
                    available_files = [f.name for f in self.base_dir.iterdir() if f.is_file()]
                    return f"❌ File '{filename}' not found. Available files: {available_files}"
                else:
                    return f"❌ Directory '{self.base_dir}' does not exist"

            # Upload the file
            bucket = self.client.bucket(self.bucket_name)
            blob = bucket.blob(filename)  # Use requested filename for blob
            blob.upload_from_filename(str(file_path))

            return f"✅ Uploaded '{actual_filename}' as '{filename}' to {self.bucket_name}"

        except Exception as e:
            return f"❌ Upload error: {str(e)}"

    @kernel_function(description="Get detailed file info")
    def get_file_info(self, filename: str) -> str:
        """Get detailed file information"""
        try:
            bucket = self.client.bucket(self.bucket_name)
            blob = bucket.blob(filename)
            if blob.exists():
                blob.reload()  # Get latest metadata
                return f"File: {filename}\\nSize: {blob.size} bytes\\nCreated: {blob.time_created}\\nType: {blob.content_type}"
            return f"File {filename} not found in bucket"
        except Exception as e:
            return f"Error getting file info: {str(e)}"

    @kernel_function(description="Check local files in directory")
    def check_local_files(self) -> str:
        """Check what files are available locally"""
        try:
            if not self.base_dir.exists():
                return f"❌ Directory {self.base_dir} does not exist"

            files = []
            for file_path in self.base_dir.iterdir():
                if file_path.is_file():
                    size = file_path.stat().st_size
                    files.append(f"{file_path.name} ({size} bytes)")

            if files:
                return f"Local files in {self.base_dir}:\\n" + "\\n".join(files)
            else:
                return f"No files found in {self.base_dir}"
        except Exception as e:
            return f"Error checking local files: {str(e)}"

# Add Enhanced GCP Agent to kernel
gcp_agent = EnhancedGCPAgent()
kernel.add_plugin(gcp_agent, plugin_name="gcp")
print("✅ Enhanced GCP Agent added to Semantic Kernel")


✅ Enhanced GCP Agent added to Semantic Kernel


## Cell 5: Define Analysis Agent

In [15]:
class AnalysisAgent:
    """Analysis Agent using Gemini 1.5 Flash"""

    def __init__(self):
        genai.configure(api_key=GEMINI_API_KEY)
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def analyze_files(self, file_data):
        """Analyze files using Gemini"""
        try:
            prompt = f"""
            You are a Data Analysis Agent. Analyze these files:

            {file_data}

            Provide:
            1. File types found
            2. Recommendations
            3. Next steps

            Keep it short and actionable.
            """

            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Analysis error: {str(e)}"

analysis_agent = AnalysisAgent()
print("✅ Analysis Agent created with Gemini 1.5 Flash")

✅ Analysis Agent created with Gemini 1.5 Flash


## Cell 6: Define Report Agent

In [16]:
class ReportAgent:
    """Report Agent using Gemini 1.5 Flash"""

    def __init__(self):
        genai.configure(api_key=GEMINI_API_KEY)
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def create_report(self, data):
        """Create report using Gemini"""
        try:
            prompt = f"""
            You are a Report Agent. Create a brief summary:

            {data}

            Provide:
            1. Summary
            2. Key findings
            3. Recommended actions

            Keep it concise and professional.
            """

            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Report error: {str(e)}"

report_agent = ReportAgent()
print("✅ Report Agent created with Gemini 1.5 Flash")

✅ Report Agent created with Gemini 1.5 Flash


## Cell 7: Define Multi-Agent Coordinator

In [17]:
class MultiAgentCoordinator:
    """Coordinates all agents using Semantic Kernel with Gemini"""

    def __init__(self, kernel):
        self.kernel = kernel

    async def run_workflow(self):
        """Run multi-agent workflow"""
        print("🚀 Multi-Agent Workflow Starting...")

        # Step 1: GCP Agent gets files (REAL Semantic Kernel)
        print("\\n📋 GCP Agent: Getting files...")
        files = await self.kernel.invoke(self.kernel.plugins["gcp"]["list_files"])
        print(f"GCP Result: {files}")

        # Step 2: Analysis Agent analyzes (using Gemini)
        print("\\n🧠 Analysis Agent: Analyzing...")
        analysis = analysis_agent.analyze_files(str(files))
        print(f"Analysis: {analysis[:100]}...")

        # Step 3: Report Agent creates report (using Gemini)
        print("\\n📊 Report Agent: Creating report...")
        report = report_agent.create_report(f"Files: {files}\\nAnalysis: {analysis}")
        print(f"Report: {report[:100]}...")

        return {
            "files": str(files),
            "analysis": analysis,
            "report": report
        }

    async def upload_and_analyze(self, filename):
        """Upload and analyze using Semantic Kernel"""
        print(f"📤 Upload and Analyze: {filename}")

        # Check local files first
        local_check = await self.kernel.invoke(self.kernel.plugins["gcp"]["check_local_files"])
        print(f"Local files check: {local_check}")

        # Upload using REAL Semantic Kernel
        upload_result = await self.kernel.invoke(
            self.kernel.plugins["gcp"]["upload_file"],
            filename=filename
        )
        print(f"Upload: {upload_result}")

        # Get files using REAL Semantic Kernel
        files = await self.kernel.invoke(self.kernel.plugins["gcp"]["list_files"])

        # Analyze with Gemini agent
        analysis = analysis_agent.analyze_files(str(files))

        return {"upload": str(upload_result), "analysis": analysis}

# Create coordinator
coordinator = MultiAgentCoordinator(kernel)
print("✅ Multi-Agent Coordinator ready")

✅ Multi-Agent Coordinator ready


## Cell 8: Define Quick Commands

In [18]:
async def quick_list():
    """Quick file list using REAL Semantic Kernel"""
    files = await kernel.invoke(kernel.plugins["gcp"]["list_files"])
    print(files)
    return files

async def quick_check_local():
    """Check local files using REAL Semantic Kernel"""
    files = await kernel.invoke(kernel.plugins["gcp"]["check_local_files"])
    print(files)
    return files

async def quick_upload(filename):
    """Quick upload using REAL Semantic Kernel"""
    result = await kernel.invoke(kernel.plugins["gcp"]["upload_file"], filename=filename)
    print(result)
    return result

async def quick_analyze():
    """Quick analysis using Gemini agents"""
    files = await kernel.invoke(kernel.plugins["gcp"]["list_files"])
    analysis = analysis_agent.analyze_files(str(files))
    print(analysis)
    return analysis

async def quick_workflow():
    """Quick full workflow"""
    return await coordinator.run_workflow()

async def quick_upload_and_analyze(filename):
    """Quick upload and analyze"""
    return await coordinator.upload_and_analyze(filename)

print("✅ Enhanced quick commands ready:")
print("- await quick_check_local()          # Check local files")
print("- await quick_list()                 # List bucket & local files")
print("- await quick_upload('filename')     # Smart upload")
print("- await quick_analyze()              # Multi-agent analysis with Gemini")
print("- await quick_workflow()             # Full workflow")
print("- await quick_upload_and_analyze('filename')  # Upload + analyze")

✅ Enhanced quick commands ready:
- await quick_check_local()          # Check local files
- await quick_list()                 # List bucket & local files
- await quick_upload('filename')     # Smart upload
- await quick_analyze()              # Multi-agent analysis with Gemini
- await quick_workflow()             # Full workflow
- await quick_upload_and_analyze('filename')  # Upload + analyze


## Cell 9: Test Enhanced System

In [19]:
print("\\n=== Testing Enhanced System with Gemini 1.5 Flash ===")

async def test_enhanced_system():
    """Test the enhanced system"""

    print("🔧 1. Checking local files...")
    local_files = await quick_check_local()

    print("\\n📋 2. Listing bucket and local files...")
    all_files = await quick_list()

    print("\\n📤 3. Testing upload with temp_eer_2.pdf...")
    upload_result = await quick_upload("temp_eer_2.pdf")

    print("\\n🧠 4. Testing analysis with Gemini 1.5 Flash...")
    analysis = await quick_analyze()

    print("\\n🚀 5. Testing full workflow with Gemini...")
    workflow = await quick_workflow()

    print("\\n🎉 ALL TESTS COMPLETE!")
    print("\\n🚀 Your Multi-Agent System with Gemini 1.5 Flash is Ready!")
    return {
        "local_files": local_files,
        "all_files": all_files,
        "upload_result": upload_result,
        "analysis": analysis,
        "workflow": workflow
    }

# Run the test
await test_enhanced_system()

\n=== Testing Enhanced System with Gemini 1.5 Flash ===
🔧 1. Checking local files...
Local files in /content:\ngen-ai-462005-7ec6dfae325a.json (2357 bytes)\ntemp_eer_2.pdf (1229101 bytes)
\n📋 2. Listing bucket and local files...
Bucket files: ['main.py']\nLocal files: ['gen-ai-462005-7ec6dfae325a.json', 'temp_eer_2.pdf']
\n📤 3. Testing upload with temp_eer_2.pdf...
✅ Uploaded 'temp_eer_2.pdf' as 'temp_eer_2.pdf' to bucket_demo8
\n🧠 4. Testing analysis with Gemini 1.5 Flash...
**1. File Types Found:** Python script (.py), JSON (.json), PDF (.pdf).

**2. Recommendations:** Investigate `main.py` to understand its function and relation to `gen-ai-462005-7ec6dfae325a.json`.  Determine if `temp_eer_2.pdf` is a duplicate across locations and consider removing the redundant copy.

**3. Next Steps:**
* Open and review `main.py`.
* Inspect `gen-ai-462005-7ec6dfae325a.json` contents.
* Compare the two `temp_eer_2.pdf` files for identical content.  Delete the redundant file if confirmed.

\n🚀 5. T

{'local_files': FunctionResult(function=KernelFunctionMetadata(name='check_local_files', plugin_name='gcp', description='Check local files in directory', parameters=[], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), value='Local files in /content:\\ngen-ai-462005-7ec6dfae325a.json (2357 bytes)\\ntemp_eer_2.pdf (1229101 bytes)', rendered_prompt=None, metadata={'arguments': {}, 'used_arguments': {}}),
 'all_files': FunctionResult(function=KernelFunctionMetadata(name='list_files', plugin_name='gcp', description='List files in bucket and local directory', parameters=[], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, 